In [9]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
from datetime import datetime,date,timedelta

In [10]:
rds_connection_string = "root:12345678@127.0.0.1/fires"
engine = create_engine(f'mysql+pymysql://{rds_connection_string}')


In [11]:
wildfire_df = pd.read_sql("select * from cleaned_wildfire_data",con=engine)
wildfire_df.head()

/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:170: Warning: (3719, "'utf8' is currently an alias for the character set UTF8MB3, but will be an alias for UTF8MB4 in a future release. Please consider using UTF8MB4 in order to be unambiguous.")
  result = self._query(query)


,level_0,index,discovery_date,containment_date,size,class,fire_year,fire_name
0,0,0,2005-02-02,2005-02-02,0.10,A,2005,FOUNTAIN
1,1,1,2004-05-12,2004-05-12,0.25,A,2004,PIGEON
2,2,2,2004-05-31,2004-05-31,0.10,A,2004,SLACK
3,3,3,2004-06-28,2004-07-03,0.10,A,2004,DEER
4,4,4,2004-06-28,2004-07-03,0.10,A,2004,STEVENOT


In [12]:
airquality_df = pd.read_sql("select * from aqi_data", con=engine)
airquality_df.head()

,date,overall_aqi,site_name,year
0,2001-01-01,153,San Francisco,2001
1,2001-01-02,140,San Francisco,2001
2,2001-01-03,140,San Francisco,2001
3,2001-01-04,107,San Francisco,2001
4,2001-01-05,124,San Francisco,2001


In [13]:
merged_data_df = airquality_df.merge(wildfire_df,left_on='date', right_on="containment_date", how='left')

In [17]:
# and write it to the database
merged_data_df.columns

Index(['date', 'overall_aqi', 'site_name', 'year', 'level_0', 'index',
       'discovery_date', 'containment_date', 'size', 'class', 'fire_year',
       'fire_name'],
      dtype='object')

In [23]:
merged_data_df.head()

,date,overall_aqi,site_name,year,level_0,index,discovery_date,containment_date,size,class,fire_year,fire_name
0,2001-01-01,153,San Francisco,2001,26892.0,26892.0,2001-01-01,2001-01-01,0.1,A,2001.0,LUCERNE
1,2001-01-01,153,San Francisco,2001,27620.0,27620.0,2001-01-01,2001-01-01,0.1,A,2001.0,DEBRIS 2
2,2001-01-01,153,San Francisco,2001,45869.0,45869.0,2001-01-01,2001-01-01,0.1,A,2001.0,LP HILL
3,2001-01-01,153,San Francisco,2001,60502.0,60502.0,2001-01-01,2001-01-01,0.1,A,2001.0,PGE 1
4,2001-01-01,153,San Francisco,2001,60589.0,60589.0,2001-01-01,2001-01-01,0.1,A,2001.0,PUMP HOUSE


In [21]:
# and write it to the database
merged_data_df.to_sql(name="fire_and_air_data", con=engine, if_exists="replace", index=False)